In [0]:
df1 = spark.read.format("json").load("dbfs:/FileStore/shared_uploads/chaitu.wynbit@outlook.com/exampleEliteJSON-1.json",multiLine=True)
df1.printSchema()

root
 |-- AgencyID: string (nullable = true)
 |-- AgencyNumber: string (nullable = true)
 |-- AuditEnabled: string (nullable = true)
 |-- CreatedByAuthor: string (nullable = true)
 |-- FinishedIncident: struct (nullable = true)
 |    |-- FinishedDateTime: string (nullable = true)
 |    |-- PerformerID: string (nullable = true)
 |-- FormHierarchyCollectionID: long (nullable = true)
 |-- IncidentIncidentNumberModValue: struct (nullable = true)
 |    |-- IncidentNumber: string (nullable = true)
 |-- IncidentOriginID: long (nullable = true)
 |-- IncidentTime: struct (nullable = true)
 |    |-- DispatchNotified: string (nullable = true)
 |    |-- IncidentTimePSAPModValue: struct (nullable = true)
 |    |    |-- PSAP: string (nullable = true)
 |-- IncidentTypeID: long (nullable = true)
 |-- Locked: boolean (nullable = true)
 |-- OID: string (nullable = true)
 |-- PatientCareReportNumber: string (nullable = true)
 |-- Scene: struct (nullable = true)
 |    |-- Exposures: array (nullable = true

In [0]:
len(flattened_df.columns)

In [0]:
print(len(set(flattened_df.columns)))

In [0]:
import pyspark
from pyspark.sql import functions as F, DataFrame
from pyspark.sql.functions import udf,flatten

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame 

from pyspark.sql.types import ArrayType, StructField, StructType



def flatten_df(df: DataFrame) -> DataFrame:

    d_list=[]
    # Compute Complex Fields in Schema   
    complex_fields = dict([(field.name, field.dataType)
            for field in df.schema.fields
            if type(field.dataType) == T.ArrayType  or  type(field.dataType) == T.StructType])

    while len(complex_fields)!= 0:
        col_name = list(complex_fields.keys())[0]
        print ("Processing :" + col_name + " Type : " + str(type(complex_fields[col_name])))

        # If StructType then convert all sub element to columns.
        if (type(complex_fields[col_name]) == T.StructType):
            expanded = [F.col(col_name + '.' + k).alias(col_name + '_' + k) for k in [ n.name for n in  complex_fields[col_name]]]
            df = df.select("*", *expanded).drop(col_name)
            # print(pivot_df)
        # print(len(pivot_df.columns))
        # 
        # If ArrayType then add the Array Elements as Rows using the explode function    
        elif (type(complex_fields[col_name]) == T.ArrayType):   
            complex_fields[col_name].needConversion()
            df = df.withColumn(col_name, F.explode_outer(col_name))
            print(len(df.columns))
            print(df.count())
            
        complex_fields = dict([(field.name, field.dataType) 
            for field in df.schema.fields
            if type(field.dataType) == T.ArrayType  or  type(field.dataType) == T.StructType])

    return df
if __name__ == "__main__":
  # Create a SparkSession.
  spark = pyspark.sql.SparkSession.builder.appName("Flatten JSON").getOrCreate()

  # Read the JSON file into a DataFrame.
  df =spark.read.format("json").load("dbfs:/FileStore/shared_uploads/chaitu.wynbit@outlook.com/exampleEliteJSON-1.json",multiLine=True)

  flattened_df = flatten_df(df)

Processing :FinishedIncident Type : <class 'pyspark.sql.types.StructType'>
Processing :IncidentIncidentNumberModValue Type : <class 'pyspark.sql.types.StructType'>
Processing :IncidentTime Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene Type : <class 'pyspark.sql.types.StructType'>
Processing :IncidentTime_IncidentTimePSAPModValue Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_Exposures Type : <class 'pyspark.sql.types.ArrayType'>
38
1
Processing :Scene_Exposures Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_Response Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_SceneAddress Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_SceneLocationTypeModValue Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_SceneNumberOfPatientsAtSceneModValue Type : <class 'pyspark.sql.types.StructType'>
Processing :Scene_Exposures_ExposureProtectiveEquipments Type : <class 'pyspark.sql.types.ArrayType'>
66
3
Proce

In [0]:
from pyspark.sql import functions as F

flattened_df = flattened_df.withColumn(
    'Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal',
    F.when(
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 3)) | 
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 8)),
        'severe'
    )
    .when(
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 9)) | 
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 12)),
        'moderate'
    )
    .when(
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 13)) | 
        (F.array_contains(F.col('Scene-Response-Patient-Vitals-VitalsGCSTotalModValue-GCSTotal'), 14)),
        'mild'
    )
    .otherwise('normal')
)


In [0]:
flattened_df.write.parquet("/Users/chaitu.wynbit@outlook.com/data/Flattened_json.parquet")

In [0]:
df.write.saveAsTable("Flattened_json_test_1")

In [0]:
print(flattened_df.Scene_Response_Patient_Vitals_VitalsGCSTotalModValue_GCSTotal)
